In [13]:
import pandas as pd
from datetime import date, timedelta, datetime
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from plotly.validators.scatter.marker import SymbolValidator
import json
from urllib.request import urlopen
import locale
locale.setlocale(locale.LC_ALL, 'pt_BR.utf8')

'pt_BR.utf8'

In [14]:
# Registrando inicio do processamento
now = datetime.now()
print("Processamento inciado às:",
      f'{now:%d/%m/%y %H:%M:%S}')

Processamento inciado às: 25/03/24 13:43:34


In [15]:
df = pd.read_parquet('../data/combustiveis_22_23.parquet')

In [16]:
df.head()

,REGIAO - SIGLA,ESTADO - SIGLA,MUNICIPIO,REVENDA,CNPJ DA REVENDA,NOME DA RUA,NUMERO RUA,COMPLEMENTO,BAIRRO,CEP,PRODUTO,DATA DA COLETA,VALOR DE VENDA,VALOR DE COMPRA,UNIDADE DE MEDIDA,BANDEIRA
0,SE,ES,COLATINA,SAO FRANCISCO DE ASSIS COMERCIO DE COMBUSTIVEI...,08.519.545/0001-10,PRACA FIDELIS FERRARI,35,None,LACE,29703-030,GASOLINA ADITIVADA,2022-07-01,"7,48",NaN,R$ / litro,VIBRA ENERGIA
1,SE,ES,COLATINA,SAO FRANCISCO DE ASSIS COMERCIO DE COMBUSTIVEI...,08.519.545/0001-10,PRACA FIDELIS FERRARI,35,None,LACE,29703-030,GASOLINA,2022-07-01,"7,38",NaN,R$ / litro,VIBRA ENERGIA
2,SE,ES,COLATINA,POSTO MOTOCAP LTDA,04.814.652/0001-10,PRACA ALMIRANTE BARROSO,52,LETRA A,LACE,29703-055,DIESEL S10,2022-07-01,"7,69",NaN,R$ / litro,VIBRA ENERGIA
3,SE,ES,COLATINA,POSTO MOTOCAP LTDA,04.814.652/0001-10,PRACA ALMIRANTE BARROSO,52,LETRA A,LACE,29703-055,ETANOL,2022-07-01,"5,99",NaN,R$ / litro,VIBRA ENERGIA
4,SE,ES,COLATINA,POSTO MOTOCAP LTDA,04.814.652/0001-10,PRACA ALMIRANTE BARROSO,52,LETRA A,LACE,29703-055,GASOLINA,2022-07-01,"7,39",NaN,R$ / litro,VIBRA ENERGIA


In [17]:
df.shape

(2674654, 16)

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2674654 entries, 0 to 2674653
Data columns (total 16 columns):
 #   Column             Dtype         
---  ------             -----         
 0   REGIAO - SIGLA     object        
 1   ESTADO - SIGLA     object        
 2   MUNICIPIO          object        
 3   REVENDA            object        
 4   CNPJ DA REVENDA    object        
 5   NOME DA RUA        object        
 6   NUMERO RUA         object        
 7   COMPLEMENTO        object        
 8   BAIRRO             object        
 9   CEP                object        
 10  PRODUTO            object        
 11  DATA DA COLETA     datetime64[us]
 12  VALOR DE VENDA     object        
 13  VALOR DE COMPRA    float64       
 14  UNIDADE DE MEDIDA  object        
 15  BANDEIRA           object        
dtypes: datetime64[us](1), float64(1), object(14)
memory usage: 326.5+ MB


In [19]:
df.columns = df.columns.str.strip().str.upper()

In [20]:
df.columns.tolist()

['REGIAO - SIGLA',
 'ESTADO - SIGLA',
 'MUNICIPIO',
 'REVENDA',
 'CNPJ DA REVENDA',
 'NOME DA RUA',
 'NUMERO RUA',
 'COMPLEMENTO',
 'BAIRRO',
 'CEP',
 'PRODUTO',
 'DATA DA COLETA',
 'VALOR DE VENDA',
 'VALOR DE COMPRA',
 'UNIDADE DE MEDIDA',
 'BANDEIRA']

In [21]:
df['DATA DA COLETA'] = pd.to_datetime(df['DATA DA COLETA'], format= '%d/%m/%Y')

In [22]:
df['DATA DA COLETA'].min()

Timestamp('2022-01-03 00:00:00')

In [23]:
df['DATA DA COLETA'].max()

Timestamp('2023-12-29 00:00:00')

In [24]:
df[
    df.duplicated(subset= df.columns.tolist(),
                 keep= False)
].shape[0]

1294728

In [25]:
antes = df.shape[0]

In [26]:
df.drop_duplicates(subset=df.columns.tolist(), inplace= True)

In [27]:
depois = df.shape[0]

In [28]:
print('Duplicadas excluídas: {}'.format(antes-depois))

Duplicadas excluídas: 863152


In [29]:
df.shape

(1811502, 16)

In [30]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1811502 entries, 0 to 2674653
Data columns (total 16 columns):
 #   Column             Dtype         
---  ------             -----         
 0   REGIAO - SIGLA     object        
 1   ESTADO - SIGLA     object        
 2   MUNICIPIO          object        
 3   REVENDA            object        
 4   CNPJ DA REVENDA    object        
 5   NOME DA RUA        object        
 6   NUMERO RUA         object        
 7   COMPLEMENTO        object        
 8   BAIRRO             object        
 9   CEP                object        
 10  PRODUTO            object        
 11  DATA DA COLETA     datetime64[us]
 12  VALOR DE VENDA     object        
 13  VALOR DE COMPRA    float64       
 14  UNIDADE DE MEDIDA  object        
 15  BANDEIRA           object        
dtypes: datetime64[us](1), float64(1), object(14)
memory usage: 235.0+ MB


In [31]:
df.head(1)

,REGIAO - SIGLA,ESTADO - SIGLA,MUNICIPIO,REVENDA,CNPJ DA REVENDA,NOME DA RUA,NUMERO RUA,COMPLEMENTO,BAIRRO,CEP,PRODUTO,DATA DA COLETA,VALOR DE VENDA,VALOR DE COMPRA,UNIDADE DE MEDIDA,BANDEIRA
0,SE,ES,COLATINA,SAO FRANCISCO DE ASSIS COMERCIO DE COMBUSTIVEI...,08.519.545/0001-10,PRACA FIDELIS FERRARI,35,None,LACE,29703-030,GASOLINA ADITIVADA,2022-07-01,"7,48",NaN,R$ / litro,VIBRA ENERGIA


In [32]:
df['ANO'] = df['DATA DA COLETA'].dt.year
df['MES'] = df['DATA DA COLETA'].dt.month

In [33]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1811502 entries, 0 to 2674653
Data columns (total 18 columns):
 #   Column             Dtype         
---  ------             -----         
 0   REGIAO - SIGLA     object        
 1   ESTADO - SIGLA     object        
 2   MUNICIPIO          object        
 3   REVENDA            object        
 4   CNPJ DA REVENDA    object        
 5   NOME DA RUA        object        
 6   NUMERO RUA         object        
 7   COMPLEMENTO        object        
 8   BAIRRO             object        
 9   CEP                object        
 10  PRODUTO            object        
 11  DATA DA COLETA     datetime64[us]
 12  VALOR DE VENDA     object        
 13  VALOR DE COMPRA    float64       
 14  UNIDADE DE MEDIDA  object        
 15  BANDEIRA           object        
 16  ANO                int32         
 17  MES                int32         
dtypes: datetime64[us](1), float64(1), int32(2), object(14)
memory usage: 248.8+ MB


In [34]:
df['NM_MES'] = (df['DATA DA COLETA'].apply(lambda x: x.strftime("%B"))).str.upper()

In [35]:
df.head(1)

,REGIAO - SIGLA,ESTADO - SIGLA,MUNICIPIO,REVENDA,CNPJ DA REVENDA,NOME DA RUA,NUMERO RUA,COMPLEMENTO,BAIRRO,CEP,PRODUTO,DATA DA COLETA,VALOR DE VENDA,VALOR DE COMPRA,UNIDADE DE MEDIDA,BANDEIRA,ANO,MES,NM_MES
0,SE,ES,COLATINA,SAO FRANCISCO DE ASSIS COMERCIO DE COMBUSTIVEI...,08.519.545/0001-10,PRACA FIDELIS FERRARI,35,None,LACE,29703-030,GASOLINA ADITIVADA,2022-07-01,"7,48",NaN,R$ / litro,VIBRA ENERGIA,2022,7,JULHO


In [36]:
df = df[df.columns.tolist()[-3:] + df.columns.tolist()[:-3]]

In [37]:
df.head(1)

,ANO,MES,NM_MES,REGIAO - SIGLA,ESTADO - SIGLA,MUNICIPIO,REVENDA,CNPJ DA REVENDA,NOME DA RUA,NUMERO RUA,COMPLEMENTO,BAIRRO,CEP,PRODUTO,DATA DA COLETA,VALOR DE VENDA,VALOR DE COMPRA,UNIDADE DE MEDIDA,BANDEIRA
0,2022,7,JULHO,SE,ES,COLATINA,SAO FRANCISCO DE ASSIS COMERCIO DE COMBUSTIVEI...,08.519.545/0001-10,PRACA FIDELIS FERRARI,35,None,LACE,29703-030,GASOLINA ADITIVADA,2022-07-01,"7,48",NaN,R$ / litro,VIBRA ENERGIA


In [38]:
df['ESTADO - SIGLA'].sort_values().unique().tolist()

['AC',
 'AL',
 'AM',
 'AP',
 'BA',
 'CE',
 'DF',
 'ES',
 'GO',
 'MA',
 'MG',
 'MS',
 'MT',
 'PA',
 'PB',
 'PE',
 'PI',
 'PR',
 'RJ',
 'RN',
 'RO',
 'RR',
 'RS',
 'SC',
 'SE',
 'SP',
 'TO']

In [39]:
df['VALOR DE VENDA'] = round(df['VALOR DE VENDA'].apply(lambda x: x.replace(',', '.')).astype(float), 2)

In [40]:
df['VALOR DE VENDA'].head()

0    7.48
1    7.38
2    7.69
3    5.99
4    7.39
Name: VALOR DE VENDA, dtype: float64

In [41]:
siglas_nomes = {
"AC": "Acre",
"AL": "Alagoas",
"AP": "Amapá",
"AM": "Amazonas",
"BA": "Bahia",
"CE": "Ceará",
"DF": "Distrito Federal",
"ES": "Espirito Santo",
"GO": "Goiás",
"MA": "Maranhão",
"MS": "Mato Grosso do Sul",
"MT": "Mato Grosso",
"MG": "Minas Gerais",
"PA": "Pará",
"PB": "Paraíba",
"PR": "Paraná",
"PE": "Pernambuco",
"PI": "Piauí",
"RJ": "Rio de Janeiro",
"RS": "Rio Grande do Sul",
"RO": "Rondônia",
"RR": "Roraima",
"SC": "Santa Catarina",
"SP": "São Paulo",
"SE": "Sergipe",
"TO": "Tocantins",
}

In [42]:
for chave in siglas_nomes.keys():
    
    df.loc[
        df['ESTADO - SIGLA'] == chave,
        'NM_ESTADO'    ] = siglas_nomes[chave].upper()

In [43]:
df.head(1)

,ANO,MES,NM_MES,REGIAO - SIGLA,ESTADO - SIGLA,MUNICIPIO,REVENDA,CNPJ DA REVENDA,NOME DA RUA,NUMERO RUA,COMPLEMENTO,BAIRRO,CEP,PRODUTO,DATA DA COLETA,VALOR DE VENDA,VALOR DE COMPRA,UNIDADE DE MEDIDA,BANDEIRA,NM_ESTADO
0,2022,7,JULHO,SE,ES,COLATINA,SAO FRANCISCO DE ASSIS COMERCIO DE COMBUSTIVEI...,08.519.545/0001-10,PRACA FIDELIS FERRARI,35,None,LACE,29703-030,GASOLINA ADITIVADA,2022-07-01,7.48,NaN,R$ / litro,VIBRA ENERGIA,ESPIRITO SANTO


In [44]:
df.columns.tolist()

['ANO',
 'MES',
 'NM_MES',
 'REGIAO - SIGLA',
 'ESTADO - SIGLA',
 'MUNICIPIO',
 'REVENDA',
 'CNPJ DA REVENDA',
 'NOME DA RUA',
 'NUMERO RUA',
 'COMPLEMENTO',
 'BAIRRO',
 'CEP',
 'PRODUTO',
 'DATA DA COLETA',
 'VALOR DE VENDA',
 'VALOR DE COMPRA',
 'UNIDADE DE MEDIDA',
 'BANDEIRA',
 'NM_ESTADO']

In [45]:
df = df[['ANO',
 'MES',
 'NM_MES',
 'REGIAO - SIGLA',
 'ESTADO - SIGLA',
 'NM_ESTADO',
 'MUNICIPIO',
 'REVENDA',
 'CNPJ DA REVENDA',
 'NOME DA RUA',
 'NUMERO RUA',
 'COMPLEMENTO',
 'BAIRRO',
 'CEP',
 'PRODUTO',
 'DATA DA COLETA',
 'VALOR DE VENDA',
 'VALOR DE COMPRA',
 'UNIDADE DE MEDIDA',
 'BANDEIRA']]

In [46]:
df_states = df.groupby([
    'ANO', 'MES', 'NM_MES', 'REGIAO - SIGLA', 'ESTADO - SIGLA', 'NM_ESTADO', 'MUNICIPIO', 'PRODUTO'
])['VALOR DE VENDA'].mean().reset_index().rename(columns= {'VALOR DE VENDA': 'VL_MEDIO_VENDA'})

In [47]:
df_states['VL_MEDIO_VENDA'] = round(df_states['VL_MEDIO_VENDA'], 2)

In [48]:
df_states.head(1)

,ANO,MES,NM_MES,REGIAO - SIGLA,ESTADO - SIGLA,NM_ESTADO,MUNICIPIO,PRODUTO,VL_MEDIO_VENDA
0,2022,1,JANEIRO,CO,DF,DISTRITO FEDERAL,BRASILIA,DIESEL,5.49


In [49]:
df_years = pd.merge(
    df.groupby(['ANO', 'PRODUTO'])['VALOR DE VENDA'].mean().reset_index().rename(columns= {'VALOR DE VENDA': 'VL_MEDIO_VENDA'}),
    df.groupby(['ANO', 'PRODUTO'])['VALOR DE VENDA'].max().reset_index().rename(columns= {'VALOR DE VENDA': 'VL_MAX_VENDA'}),
    on= ['ANO', 'PRODUTO'],
    how= 'left'
).merge(
    df.groupby(['ANO', 'PRODUTO'])['VALOR DE VENDA'].min().reset_index().rename(columns= {'VALOR DE VENDA': 'VL_MIN_VENDA'}),
    on= ['ANO', 'PRODUTO'],
    how= 'left'
    )

In [50]:
df.head(1)

,ANO,MES,NM_MES,REGIAO - SIGLA,ESTADO - SIGLA,NM_ESTADO,MUNICIPIO,REVENDA,CNPJ DA REVENDA,NOME DA RUA,NUMERO RUA,COMPLEMENTO,BAIRRO,CEP,PRODUTO,DATA DA COLETA,VALOR DE VENDA,VALOR DE COMPRA,UNIDADE DE MEDIDA,BANDEIRA
0,2022,7,JULHO,SE,ES,ESPIRITO SANTO,COLATINA,SAO FRANCISCO DE ASSIS COMERCIO DE COMBUSTIVEI...,08.519.545/0001-10,PRACA FIDELIS FERRARI,35,None,LACE,29703-030,GASOLINA ADITIVADA,2022-07-01,7.48,NaN,R$ / litro,VIBRA ENERGIA


In [51]:
df_years = pd.merge(
    df_years,
    df.groupby(['ANO', 'PRODUTO'])['REVENDA'].count().reset_index().rename(columns= {'REVENDA': 'QT_COLETADO'})
)

In [52]:
df_years

,ANO,PRODUTO,VL_MEDIO_VENDA,VL_MAX_VENDA,VL_MIN_VENDA,QT_COLETADO
0,2022,DIESEL,6.631457,9.00,3.14,102477
1,2022,DIESEL S10,6.734393,9.65,3.59,184760
2,2022,ETANOL,4.851732,7.98,2.49,201350
3,2022,GASOLINA,6.317327,8.99,3.49,228810
4,2022,GASOLINA ADITIVADA,6.472456,9.28,4.09,172855
5,2022,GNV,5.042833,7.99,3.17,17250
6,2023,DIESEL,5.758133,7.99,3.97,106136
7,2023,DIESEL S10,5.859705,9.00,4.19,168461
8,2023,ETANOL,4.004911,6.96,2.69,197849
9,2023,GASOLINA,5.528898,8.19,4.09,232085


In [53]:
df_years['VL_MEDIO_VENDA'] = round(df_years['VL_MEDIO_VENDA'], 2)

In [54]:
df_years

,ANO,PRODUTO,VL_MEDIO_VENDA,VL_MAX_VENDA,VL_MIN_VENDA,QT_COLETADO
0,2022,DIESEL,6.63,9.00,3.14,102477
1,2022,DIESEL S10,6.73,9.65,3.59,184760
2,2022,ETANOL,4.85,7.98,2.49,201350
3,2022,GASOLINA,6.32,8.99,3.49,228810
4,2022,GASOLINA ADITIVADA,6.47,9.28,4.09,172855
5,2022,GNV,5.04,7.99,3.17,17250
6,2023,DIESEL,5.76,7.99,3.97,106136
7,2023,DIESEL S10,5.86,9.00,4.19,168461
8,2023,ETANOL,4.00,6.96,2.69,197849
9,2023,GASOLINA,5.53,8.19,4.09,232085


In [55]:
df_states

,ANO,MES,NM_MES,REGIAO - SIGLA,ESTADO - SIGLA,NM_ESTADO,MUNICIPIO,PRODUTO,VL_MEDIO_VENDA
0,2022,1,JANEIRO,CO,DF,DISTRITO FEDERAL,BRASILIA,DIESEL,5.49
1,2022,1,JANEIRO,CO,DF,DISTRITO FEDERAL,BRASILIA,DIESEL S10,5.61
2,2022,1,JANEIRO,CO,DF,DISTRITO FEDERAL,BRASILIA,ETANOL,5.81
3,2022,1,JANEIRO,CO,DF,DISTRITO FEDERAL,BRASILIA,GASOLINA,6.80
4,2022,1,JANEIRO,CO,DF,DISTRITO FEDERAL,BRASILIA,GASOLINA ADITIVADA,6.88
...,...,...,...,...,...,...,...,...,...
52221,2023,12,DEZEMBRO,SE,SP,SÃO PAULO,VOTUPORANGA,DIESEL,5.86
52222,2023,12,DEZEMBRO,SE,SP,SÃO PAULO,VOTUPORANGA,DIESEL S10,6.06
52223,2023,12,DEZEMBRO,SE,SP,SÃO PAULO,VOTUPORANGA,ETANOL,3.06
52224,2023,12,DEZEMBRO,SE,SP,SÃO PAULO,VOTUPORANGA,GASOLINA,5.49


In [56]:
df.columns

Index(['ANO', 'MES', 'NM_MES', 'REGIAO - SIGLA', 'ESTADO - SIGLA', 'NM_ESTADO',
       'MUNICIPIO', 'REVENDA', 'CNPJ DA REVENDA', 'NOME DA RUA', 'NUMERO RUA',
       'COMPLEMENTO', 'BAIRRO', 'CEP', 'PRODUTO', 'DATA DA COLETA',
       'VALOR DE VENDA', 'VALOR DE COMPRA', 'UNIDADE DE MEDIDA', 'BANDEIRA'],
      dtype='object')

In [57]:
df_comparativo_geral = df.groupby([
    'ANO', 'MES', 'NM_MES', 'PRODUTO'
])['VALOR DE VENDA'].mean().reset_index().rename(columns= {'VALOR DE VENDA': "VL_MEDIO_VENDA"})

In [58]:
df_comparativo_states = df.groupby([
    'ANO', 'MES', 'NM_MES', 'PRODUTO', 'ESTADO - SIGLA', 'NM_ESTADO'
])['VALOR DE VENDA'].mean().reset_index().rename(columns= {'VALOR DE VENDA': "VL_MEDIO_VENDA"})

In [59]:
df_comparativo_citys = df.groupby([
    'ANO', 'MES', 'NM_MES', 'PRODUTO', 'NM_ESTADO', 'MUNICIPIO'
])['VALOR DE VENDA'].mean().reset_index().rename(columns= {'VALOR DE VENDA': "VL_MEDIO_VENDA"})

In [60]:
df_comparativo_geral['VL_MEDIO_VENDA'] = df_comparativo_geral['VL_MEDIO_VENDA'].apply(lambda x: round(x, 2))
df_comparativo_states['VL_MEDIO_VENDA'] = df_comparativo_states['VL_MEDIO_VENDA'].apply(lambda x: round(x, 2))
df_comparativo_citys['VL_MEDIO_VENDA'] = df_comparativo_citys['VL_MEDIO_VENDA'].apply(lambda x: round(x, 2))

In [61]:
df_comparativo_geral.head()

,ANO,MES,NM_MES,PRODUTO,VL_MEDIO_VENDA
0,2022,1,JANEIRO,DIESEL,5.51
1,2022,1,JANEIRO,DIESEL S10,5.57
2,2022,1,JANEIRO,ETANOL,5.26
3,2022,1,JANEIRO,GASOLINA,6.64
4,2022,1,JANEIRO,GASOLINA ADITIVADA,6.79


In [62]:
df_comparativo_states.head()

,ANO,MES,NM_MES,PRODUTO,ESTADO - SIGLA,NM_ESTADO,VL_MEDIO_VENDA
0,2022,1,JANEIRO,DIESEL,AC,ACRE,6.33
1,2022,1,JANEIRO,DIESEL,AL,ALAGOAS,5.77
2,2022,1,JANEIRO,DIESEL,AM,AMAZONAS,5.66
3,2022,1,JANEIRO,DIESEL,AP,AMAPÁ,5.53
4,2022,1,JANEIRO,DIESEL,BA,BAHIA,5.70


In [63]:
df_comparativo_citys.head()

,ANO,MES,NM_MES,PRODUTO,NM_ESTADO,MUNICIPIO,VL_MEDIO_VENDA
0,2022,1,JANEIRO,DIESEL,ACRE,CRUZEIRO DO SUL,6.68
1,2022,1,JANEIRO,DIESEL,ACRE,RIO BRANCO,6.19
2,2022,1,JANEIRO,DIESEL,ALAGOAS,ARAPIRACA,5.48
3,2022,1,JANEIRO,DIESEL,ALAGOAS,DELMIRO GOUVEIA,6.12
4,2022,1,JANEIRO,DIESEL,ALAGOAS,MACEIO,5.57


# Mapa

# geojson

In [64]:
geojson = json.load(open('../data/brasil_estados.json'))

# df_map

In [65]:
df_map = df.groupby([
    'PRODUTO', 'ESTADO - SIGLA', 'NM_ESTADO'
])['VALOR DE VENDA'].mean().reset_index().rename(columns= {'VALOR DE VENDA': "VL_MEDIO_VENDA"})

In [66]:
df_map['VL_MEDIO_VENDA'] = df_map['VL_MEDIO_VENDA'].apply(lambda x: round(x, 2))

In [ ]:
produtos = df['PRODUTO'].unique().tolist()

for produto in produtos:

    # Mapa
    map_graph = go.Figure()

    map_graph.add_trace(go.Choroplethmapbox(
        geojson= geojson,
        locations= df_map[df_map['PRODUTO'] == produto]['ESTADO - SIGLA'],
        z= df_map[df_map['PRODUTO'] == produto]['VL_MEDIO_VENDA'],
        colorscale= 'haline',
        colorbar= dict(title='Valor Médio de Venda',
                    title_font_color= 'black',
                    title_font_size= 20,
                    tickprefix='R$ ',
                    tickfont_color= 'black',
                    x= 0.1
                    ),
        hovertext= df_map[df_map['PRODUTO'] == produto]['NM_ESTADO'] + '<br>' + 'R$ ' + df_map[df_map['PRODUTO'] == produto]['VL_MEDIO_VENDA'].apply(lambda x: str(x)),
        hoverinfo= 'text'
    ))


    # Layout do mapa
    map_graph.update_layout(
        mapbox_style= 'white-bg',
        mapbox_zoom= 3,
        mapbox_center ={'lat': -14, 'lon': -55},
        margin= dict(l=0, r=0, t=0, b=0),
        geo= dict(lakecolor='rgb(255, 255, 255)'),
        height= 625,
        hoverlabel= dict(
                font= dict(color= 'white',
                            size= 14),
                bgcolor= 'black')
    )

    map_graph.show()

In [67]:
df_map.to_csv('../data/df_map.csv',
             sep= ';',
             index= False)

In [68]:
df_comparativo_geral.to_csv('../data/df_comparativo_geral.csv',
               sep= ';',
                    index= False)

In [69]:
df_comparativo_states.to_csv('../data/df_comparativo_states.csv',
               sep= ';',
                    index= False)

In [70]:
df_comparativo_citys.to_csv('../data/df_comparativo_citys.csv',
               sep= ';',
                    index= False)

In [71]:
df_states.to_csv('../data/df_states.csv',
               sep= ';',
               index= False)

In [72]:
df_years.to_csv('../data/df_years.csv',
               sep= ';',
               index= False)

In [73]:
# Registrando término do processamento
now = datetime.now()
print('Processamento finalizado em: {:%d/%m/%Y %H:%M:%S}'.format(now))

Processamento finalizado em: 25/03/2024 13:44:18
